In [ ]:
import os

import numpy as np
import pandas as pd

from shapely.geometry import LineString
from shapely.geometry import Point, MultiPoint, MultiLineString
from shapely.ops import linemerge, nearest_points

import geopandas as gpd
from geopandas import GeoDataFrame
from centerline.geometry import Centerline

from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

## Get Sidewalk Centerlines

In [ ]:
df = gpd.read_file('./data/pedestrian.shp')

In [ ]:
crs = {'init': 'epsg:6707'}
df.crs = crs

In [ ]:
df.plot(figsize=(12, 12), cmap='viridis')

In [ ]:
df_dissolved = gpd.GeoDataFrame(geometry=gpd.GeoSeries([geom for geom in df.unary_union.geoms]))

In [ ]:
df_exploded = gpd.GeoDataFrame(df_dissolved.geometry.explode())

In [ ]:
df_exploded.plot(figsize=(12, 12), cmap='viridis')

In [ ]:
df_exploded['centerlines'] = df_exploded.progress_apply(lambda row: Centerline(row.geometry), axis=1)

In [ ]:
df_exploded = df_exploded.set_geometry('centerlines')

In [ ]:
df_exploded.plot(figsize=(12, 12), cmap='viridis')

## Remove Short Line Ends

In [ ]:
df_exploded['centerlines'] = df_exploded['centerlines'].progress_apply(linemerge)

In [ ]:
def remove_short_lines(line):
    
    if line.type == 'MultiLineString':
        
        passing_lines = []
    
        for i, linestring in enumerate(line):
            
            other_lines = MultiLineString([x for j, x in enumerate(line) if j != i])
            
            p0 = Point(linestring.coords[0])
            p1 = Point(linestring.coords[-1])
            
            is_deadend = False
            
            if p0.disjoint(other_lines): is_deadend = True
            if p1.disjoint(other_lines): is_deadend = True
            
            if not is_deadend or linestring.length > 5:                
                passing_lines.append(linestring)
            
        return MultiLineString(passing_lines)
            
    if line.type == 'LineString':
        return line

In [ ]:
df_exploded['centerlines'] = df_exploded['centerlines'].progress_apply(remove_short_lines)

In [ ]:
df_exploded.head()

In [ ]:
df_exploded.plot(figsize=(12, 12), cmap='viridis')

## Get Sidewalk Widths

In [ ]:
df_exploded['centerlines'] = df_exploded['centerlines'].progress_apply(lambda row: row.simplify(1, preserve_topology=True))

In [ ]:
df_exploded.plot(figsize=(12, 12), cmap='viridis')

In [ ]:
def linestring_to_segments(linestring):
    return [LineString([linestring.coords[i], linestring.coords[i+1]]) for i in range(len(linestring.coords) - 1)]

In [ ]:
def get_segments(line):
    
    line_segments = []

    if line.type == 'MultiLineString':
        
        for linestring in line.geoms:
            
            line_segments.extend(linestring_to_segments(linestring))

    if line.type == 'LineString':
        
        line_segments.extend(linestring_to_segments(line))

    return line_segments

In [ ]:
df_exploded.head(1)

In [ ]:
df_exploded['segments'] = df_exploded['centerlines'].progress_apply(get_segments)
df_exploded.head(1)

In [ ]:
def interpolate_by_distance(linestring):
    
    distance = 1
    all_points = []
    count = round(linestring.length / distance) + 1
    
    if count == 1:
        all_points.append(linestring.interpolate(linestring.length / 2))
    
    else:
        for i in range(count):
            all_points.append(linestring.interpolate(distance * i))
    
    return all_points

def interpolate(line):
    
    if line.type == 'MultiLineString':
        
        all_points = []
        
        for linestring in line:
            all_points.extend(interpolate_by_distance(linestring))
        
        return MultiPoint(all_points)
            
    if line.type == 'LineString':
        return MultiPoint(interpolate_by_distance(line))
    
    
def polygon_to_multilinestring(polygon):

    return MultiLineString([polygon.exterior] + [line for line in polygon.interiors])
    

def get_avg_distances(row):
    
    avg_distances = []
    
    sidewalk_lines = polygon_to_multilinestring(row.geometry)
    
    for segment in row.segments:
        
        points = interpolate(segment)
        
        distances = []
        
        for point in points:
            p1, p2 = nearest_points(sidewalk_lines, point)
            distances.append(p1.distance(p2))
            
        avg_distances.append(sum(distances) / len(distances))
        
    return avg_distances

In [ ]:
df_exploded['avg_distances'] = df_exploded.progress_apply(lambda row: get_avg_distances(row), axis=1)
df_exploded.head(1)

In [ ]:
data = {'geometry': [], 'width': []}

for i, row in df_exploded.iterrows():
    
    for segment in row.segments:
        data['geometry'].append(segment)
    
    for distance in row.avg_distances:
        data['width'].append(distance * 2)
        
df_segments = pd.DataFrame(data)
df_segments = GeoDataFrame(df_segments, crs=crs, geometry='geometry')
df_segments.head(1)

In [ ]:
df_segments.plot(figsize=(12, 12), cmap='viridis')

In [ ]:
df_segments.plot(figsize=(12, 12), column='width', cmap='viridis')

In [ ]:
df_segments.head()

In [ ]:
df_segments.crs

In [ ]:
with open('./sidewalkwidths.geojson', 'w') as f:
    f.write(df_segments.to_json())